# Import Data and Save as JSON

### Load Pickle Data

In [1]:
import pandas as pd
from pathlib import Path

data_dir = Path('/mnt/data/factcheck/tweets/similarity_web_data')

print(pd.__version__)
df = pd.read_pickle(data_dir / 'df_titles.pkl')

1.4.2


### Inspect data

In [2]:
pd.set_option("display.max_columns", None)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

def between_incl(yl, ml, dl, yh, mh, dh):
    low_date = pd.Timestamp(yl, ml, dl)
    high_date = pd.Timestamp(yh, mh, dh)
    return df[(df['date'] >= low_date) & (df['date'] <= high_date)]

df.head()

id      conversation_id                created_at  \
0  1422203548138450949  1422203548138450949  2021-08-02 16:32:03 CEST   
1  1422196620150648840  1422196620150648840  2021-08-02 16:04:32 CEST   
2  1422186538641862658  1422186538641862658  2021-08-02 15:24:28 CEST   
3  1422179557759721472  1422179548381302790  2021-08-02 14:56:44 CEST   
4  1422179548381302790  1422179548381302790  2021-08-02 14:56:41 CEST   

        date      time timezone     user_id         username  \
0 2021-08-02  16:32:03    +0200  1408164626  alenaschillerov   
1 2021-08-02  16:04:32    +0200  1408164626  alenaschillerov   
2 2021-08-02  15:24:28    +0200  1408164626  alenaschillerov   
3 2021-08-02  14:56:44    +0200  1408164626  alenaschillerov   
4 2021-08-02  14:56:41    +0200  1408164626  alenaschillerov   

                name place                                              tweet  \
0  Alena Schillerová        Dnes je poslední den pro podání žádosti o nový...   
1  Alena Schillerová        Více než 70 procent dospělé populace v EU bylo...   
2  Alena Schillerová        Je tu NOVÝ TÝDEN a s ním důležitá zpráva. Česk...   
3  Alena Schillerová        Celkem byla k 31. červenci 2021 na podporu eko...   
4  Alena Schillerová        Pokračuje i vládní pomoc ekonomice na zmírnění...   

  language                                           mentions urls  \
0       cs  [{'screen_name': 'financnisprava', 'name': 'fi...   []   
1       cs                                                 []   []   
2       cs                                                 []   []   
3       cs                                                 []   []   
4       cs                                                 []   []   

                                              photos  replies_count  \
0                                                 []              6   
1                                                 []             12   
2                                                 []             72   
3  [https://pbs.twimg.com/media/E7yXwFqXIAc1BdW.jpg]              2   
4  [https://pbs.twimg.com/media/E7yXBUoXIAM6j-a.jpg]              5   

   retweets_count  likes_count hashtags cashtags  \
0               2           10       []       []   
1               0           15       []       []   
2               6           34       []       []   
3               3            7       []       []   
4               2           16       []       []   

                                                link  retweet  \
0  https://twitter.com/alenaschillerov/status/142...    False   
1  https://twitter.com/alenaschillerov/status/142...    False   
2  https://twitter.com/alenaschillerov/status/142...    False   
3  https://twitter.com/alenaschillerov/status/142...    False   
4  https://twitter.com/alenaschillerov/status/142...    False   

                                           quote_url  video  \
0  https://twitter.com/financnisprava/status/1422...      0   
1  https://twitter.com/EU_Commission/status/14215...      0   
2                                                         1   
3                                                         1   
4                                                         1   

                                         thumbnail near geo source user_rt_id  \
0                                                                               
1                                                                               
2  https://pbs.twimg.com/media/E7yfb32WEAoGncr.jpg                              
3  https://pbs.twimg.com/media/E7yXwFqXIAc1BdW.jpg                              
4  https://pbs.twimg.com/media/E7yXBUoXIAM6j-a.jpg                              

  user_rt retweet_id reply_to retweet_date translate trans_src trans_dest  \
0                          []                                               
1                          []                                               
2                          []                               

In [3]:
for year in range(2015, 2023):
    sub = between_incl(year, 1, 1, year, 12, 31)
    print(year, len(sub))
    df[year] = len(sub)

2015 10918
2016 19646
2017 23553
2018 38356
2019 34840
2020 38738
2021 25451
2022 4052


In [4]:
print('min', df.date.min(), '  max', df.date.max())

min 2009-02-12 00:00:00   max 2022-03-07 00:00:00


In [5]:
print(len(between_incl(2015, 1, 1, 2025, 1, 1)))
print(len(between_incl(2014, 1, 1, 2018, 12, 31)))

195554
97359


In [6]:
print(len(df))

203106


# Filter Tweets into the sub Variable And Save to Train and Test Datasets

In [3]:
import json

def make_set(dates, name_id_to_text, name_id_to_vecno):
    sub = between_incl(*dates)[['id', 'tweet']]
    sub = sub.rename(columns={'tweet': 'text'})
    id_to_text = {id_: text for _, (id_, text) in sub.iterrows()}
    id_to_vecno = {id_: e for e, (_, (id_,text)) in enumerate(sub.iterrows())}
    print(len(id_to_text), len(id_to_vecno))
    for k, v in id_to_text.items():
        print(k, v)
        break
    for k, v in id_to_vecno.items():
        print(k, v)
        break
    with open(name_id_to_text, 'wt', encoding='utf-8') as f:
        json.dump(id_to_text, f, ensure_ascii=False)
    with open(name_id_to_vecno, 'wt', encoding='utf-8') as f:
        json.dump(id_to_vecno, f, ensure_ascii=False)
        
make_set([2014, 1, 1, 2018, 12, 31], data_dir / 'train_text.json', data_dir / 'train_vecno.json')
make_set([2019, 1, 1, 2030, 12, 31], data_dir / 'test_text.json', data_dir / 'test_vecno.json')
make_set([2010, 1, 1, 2030, 12, 31], data_dir / 'all_text.json', data_dir / 'all_vecno.json')

for flnm in data_dir / 'train_text.json', data_dir / 'train_vecno.json':
    with open(flnm, 'rt') as f:
        for i in f:
            print(i[:1000])
            break

97359 97359
1076886878660497408 Andrej Babiš a jeho vánoční přání. Dozvíte se mimo jiné, jaký vánoční dárek má pan premiér nejradši. Tak snad se mu přání vyplní. Krásné sváteční dny plné splněných snů vám všem!
1076886878660497408 0
103081 103081
1422203548138450949 Dnes je poslední den pro podání žádosti o nový kompenzační bonus. Dosud @financnisprava vyplatila živnostníkům a dohodářům pomoc za více než 14 miliard korun 👍
1422203548138450949 0
203057 203057
1422203548138450949 Dnes je poslední den pro podání žádosti o nový kompenzační bonus. Dosud @financnisprava vyplatila živnostníkům a dohodářům pomoc za více než 14 miliard korun 👍
1422203548138450949 0
{"1076886878660497408": "Andrej Babiš a jeho vánoční přání. Dozvíte se mimo jiné, jaký vánoční dárek má pan premiér nejradši. Tak snad se mu přání vyplní. Krásné sváteční dny plné splněných snů vám všem!", "1076108256152571905": "Daňový balíček zavádí také zdanění zahřívaného tabáku. Není žádný důvod, aby zahřívaný tabák na rozdíl od

### Wrap Text

In [8]:
import textwrap
import json

def wrap(text):
    return textwrap.fill(text, width=50).replace("\n", "<br>")

def make_wrap(source, dest):
    with open(source, 'rt') as f:
        train_to_text = json.load(f)

    train_to_wrap = {k: wrap(v) for k, v in train_to_text.items()}
    print(next(iter(train_to_wrap.values())))
    with open(dest, 'wt', encoding='utf-8') as f:
        json.dump(train_to_wrap, f, ensure_ascii=False)    
        
make_wrap(data_dir / 'train_text.json', data_dir / 'train_wrap.json')
make_wrap(data_dir / 'all_text.json', data_dir / 'all_wrap.json')

Andrej Babiš a jeho vánoční přání. Dozvíte se mimo<br>jiné, jaký vánoční dárek má pan premiér nejradši.<br>Tak snad se mu přání vyplní. Krásné sváteční dny<br>plné splněných snů vám všem!
Dnes je poslední den pro podání žádosti o nový<br>kompenzační bonus. Dosud @financnisprava vyplatila<br>živnostníkům a dohodářům pomoc za více než 14<br>miliard korun 👍


In [13]:
import json

with (data_dir / 'all_wrap.json').open('rt') as f:
    js = json.load(f)

js['1374316416522944512']
# '1374316416522944512'

'Mediální válka proti a popírání účinnosti<br>Ivermektinu je vědomá GENOCIDA!!!  Odmítání léčby<br>Ivermektinem je VRAŽDA a “lékař-ka”, kteří<br>odmítnou použit Ivermektin, když jej mají k<br>dispozici jsou MASOVÍ VRAZI na úrovni Mengeleho.'

# Compute Vectors

### Initial Imports

In [24]:
from pathlib import Path
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from tqdm.auto import tqdm
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


### Download, Save and Load Model into the Memory

In [5]:
tok_mod_path = Path('/mnt/data/factcheck/tweets/similarity/models')

models_full_names = [
    'AIDA-UPM/mstsb-paraphrase-multilingual-mpnet-base-v2',
    'AIDA-UPM/MSTSb_paraphrase-multilingual-MiniLM-L12-v2',
    'AIDA-UPM/MSTSb_stsb-xlm-r-multilingual',
    'AIDA-UPM/MSTSb_paraphrase-xlm-r-multilingual-v1'
]

models_simple_names = [
    'para_mpnet',
    'para_minilm',
    'stsb_xlm',
    'para_xlm'
]

def get_tokenizer_path(name):
    return tok_mod_path / f'{name}_tokenizer'

def get_model_path(name):
    return tok_mod_path / f'{name}_model'

In [6]:
save_models = False

if save_models:
    from transformers import AutoTokenizer, AutoModel

    for full_name, simple_name in zip(models_full_names, models_simple_names):
        tokenizer = AutoTokenizer.from_pretrained(full_name)
        model = AutoModel.from_pretrained(full_name)
        
        tokenizer_path = get_tokenizer_path(simple_name)
        tokenizer.save_pretrained(tokenizer_path)
        
        model_path = get_model_path(simple_name)
        model.save_pretrained(model_path)

### Load Files

In [2]:
import json

def load_dict(flnm):
    with open(flnm, 'rt') as f:
        data = json.load(f)
    data = {int(k): v for k, v in data.items()}
    return data

all_to_text = load_dict(data_dir / 'all_text.json')
all_to_wrap = load_dict(data_dir / 'all_wrap.json')
all_to_vecno = load_dict(data_dir / 'all_vecno.json')
train_to_text = load_dict(data_dir / 'train_text.json')
train_to_wrap = load_dict(data_dir / 'train_wrap.json')
train_to_vecno = load_dict(data_dir / 'train_vecno.json')
test_to_text = load_dict(data_dir / 'test_text.json')

print(len(train_to_text), len(train_to_wrap), len(train_to_vecno))
print(len(all_to_text), len(all_to_wrap), len(all_to_vecno))
print(len(test_to_text))

97359 97359 97359
203057 203057 203057
103081


### Compute and Save Vectors

In [ ]:
BATCH_SIZE = 1024  # please, reduce if memory exhausted

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


from torch.utils.data import DataLoader, TensorDataset

def vectorize_and_save(claims, tokenizer_path, model_path, out_flnm):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    model = AutoModel.from_pretrained(model_path)
    model.to(device)
    model.eval()
    print('loaded')
    
    encoded_claims = tokenizer(claims, padding=True, truncation=True, return_tensors='pt')
    print('Tokenized', encoded_claims.keys())

    ds = TensorDataset(*encoded_claims.values())
    dl = DataLoader(dataset=ds, batch_size=BATCH_SIZE, shuffle=False)

    tq = tqdm(range(len(dl)))
    
    res = None

    with torch.no_grad():
        for batch in dl:        
            tq.update(1)
            batch = [item.to(device) for item in batch]
            batch_dict = {name: data for name, data in zip(encoded_claims.keys(), batch)}
            model_output = model(**batch_dict)
            out = mean_pooling(model_output, batch_dict['attention_mask']).to('cpu')

            res = out if res is None else torch.cat((res, out))

    print('\nShape output:', res.shape, '  Len input:', len(claims))
    torch.save(res, out_flnm)

claims = list(all_to_text.values())
for i, name in enumerate(models_simple_names):
    tokenizer_path = get_tokenizer_path(name)
    model_path = get_model_path(name)
    vectorize_and_save(claims, tokenizer_path, model_path, data_dir / f'all_tensors_{i}.pt')

### Join and Save Vectors

In [12]:
tensors = []
for i in range(4):
    tensor = torch.load(data_dir / f'all_tensors_{i}.pt')
    tensors.append(tensor)

In [13]:
print(*[tensors[i].shape for i in range(4)])
concatenated = torch.cat(tensors, dim=-1)
print(concatenated.shape)
torch.save(concatenated, data_dir / 'all_tensors.pt')

torch.Size([203057, 768]) torch.Size([203057, 384]) torch.Size([203057, 768]) torch.Size([203057, 768])
torch.Size([203057, 2688])


### Load Vectors, Compute PCA and Save

In [14]:
import torch
import numpy as np

data = torch.load(data_dir / 'all_tensors.pt').numpy()

In [15]:
from sklearn.decomposition import PCA

print(data.shape)
pca = PCA(n_components=512)
pca.fit(data)
res = pca.transform(data)
t = torch.tensor(res)
print(t.shape, t.dtype)
torch.save(t, data_dir / 'all_tensors_pca.pt')

(203057, 2688)
torch.Size([203057, 512]) torch.float32
